In [15]:
#---Implementation of the Project of Artificial Intelligence 
#--- i160164 - Usman Ali & i170130  - Abubakr Idrees
#--- KNN with sparse matrix for item based recommendation

#-- commenter link - https://colab.research.google.com/drive/1N_d-4gdUJGWrUTIVSFNpPV-8rAi-X0yf?usp=sharing

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [16]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd 
r_cols = ['user_id', 'item_id', 'rating', 'timestamp']
dataset = pd.read_csv('/content/drive/MyDrive/AI_Project/DataSet/u.data', sep='\t', names=r_cols, encoding='latin-1')

m_cols = ['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/drive/MyDrive/AI_Project/DataSet/u.item', sep='|', names=m_cols, usecols=range(5), encoding='latin-1')

movies.drop(movies.columns[[3,4]], inplace = True, axis = 1 ) #to print titles than urls



In [18]:
n_users = dataset.user_id.unique().shape[0]
n_items = dataset.item_id.unique().shape[0]
n_items = dataset['item_id'].max()
RatingMatrix = np.zeros((n_users,n_items))
for line in dataset.itertuples():           #iterat over name tuples
    RatingMatrix[line[1]-1,line[2]-1] = line[3]
print("Original rating matrix : ",RatingMatrix)

csr_sample = csr_matrix(RatingMatrix) #user to movie ratings
#print(csr_sample) 

Original rating matrix :  [[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [19]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=3, n_jobs=-1)
knn.fit(csr_sample)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=3, p=2, radius=1.0)

In [20]:
dataset_sort_des = dataset.sort_values(['user_id', 'timestamp'], ascending=[True, False])
recommendation = dataset_sort_des[dataset_sort_des['user_id'] == 3].item_id
recommendation = recommendation.tolist()
recommendation = recommendation[:20]
print("Items liked by user: ",recommendation)

Items liked by user:  [318, 320, 317, 181, 331, 328, 348, 327, 321, 260, 329, 347, 340, 346, 351, 330, 264, 338, 325, 335]


In [21]:
distances1=[]
ListTORecommend=[]
for i in recommendation:
  distances , indices = knn.kneighbors(csr_sample[i],n_neighbors=3)
  indices = indices.flatten()
  indices= indices[1:]
  ListTORecommend.extend(indices)
print("Items to be recommended: ",ListTORecommend)

[movies.loc[movies['item_id'].isin(ListTORecommend)]]

Items to be recommended:  [919, 240, 17, 408, 193, 270, 702, 688, 221, 346, 833, 432, 517, 905, 787, 550, 709, 869, 509, 169, 377, 863, 937, 180, 510, 782, 221, 520, 420, 7, 5, 746, 758, 705, 560, 473, 649, 312, 475, 789]


[     item_id                                              title release_date
 4          5                                     Copycat (1995)  01-Jan-1995
 6          7                              Twelve Monkeys (1995)  01-Jan-1995
 16        17                         From Dusk Till Dawn (1996)  05-Feb-1996
 168      169                         Wrong Trousers, The (1993)  01-Jan-1993
 179      180                              Apocalypse Now (1979)  01-Jan-1979
 192      193                            Right Stuff, The (1983)  01-Jan-1983
 220      221                          Breaking the Waves (1996)  15-Nov-1996
 239      240             Beavis and Butt-head Do America (1996)  20-Dec-1996
 269      270                                     Gattaca (1997)  01-Jan-1997
 311      312     Midnight in the Garden of Good and Evil (1997)  01-Jan-1997
 345      346                                Jackie Brown (1997)  01-Jan-1997
 376      377                                Heavyweights (1994)